In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules -------------------------------------------------------------------------------------
import pyfesom2 as pf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from netCDF4 import Dataset

import time
from datetime import date

#import sys
#sys.path.append('home/ollie/mozeisin/SkillMetrics')

import cartopy.crs as ccrs
#import pangaeapy.pandataset as pd

# own py_recom modules ----------------------------------------------------------------------
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

#from load_interp_WOA_python3 import WOAdata
from plot_Taylor_normalized import plt_Taylor_norm

In [ ]:
# After changing these parameters here, save the notebook and clear the output. 
# Then run this cell and the cells with the desired parameters.

# run specification -------------------------------------------------------------------------------------
runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/tep1' 
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis 
first_year = 1950
last_year  = 1980
    
years = [first_year, last_year]

# specification of analysis ------------------------------------------------------------------------------------
depth = 5
layerwise = False
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# export of analysis ------------------------------------------------------------------------------------
today = date.today().strftime("_%Y_%m_%d")

savepath = '/home/ollie/mozeisin/evaluation/mo_files/tep1/'
savefig = False
htmlname     =  'tep1' + today +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'
    

In [ ]:
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)

# Chlorophyll

In [ ]:
!jupyter nbconvert --to=python Chlsurf_OCCCI_Comparison_python3.ipynb
from Chlsurf_OCCCI_Comparison_python3 import Chlsurf_OCCCI_comp

# OCCCI   
matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'

# analyze only last 10 years
test = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          last_year-10,last_year,
                          #first_year,last_year,
                          mapproj=mapproj,savefig=savefig)


# Intracellular carbon

In [ ]:
# Diatom carbon

NCfesom = resultpath + '/DiaC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Diatom intracell. Carbon {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'intracellular C [mmol C m$^{-3}$]'

DiaC = pf.get_data(resultpath, "DiaC", years=1950, mesh=mesh, runid=runid, how="mean", depth=depth)
fig = pf.plot(mesh, DiaC, mapproj = mapproj, units=units, titles=label)


In [ ]:
# Phytplankton carbon

NCfesom = resultpath + '/PhyC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Phytoplankton intracell. Carbon {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'intracellular C [mmol C m$^{-3}$]'

PhyC = pf.get_data(resultpath, "PhyC", years=1950, mesh=mesh, runid=runid, how="mean", depth = depth)
fig = pf.plot(mesh, PhyC, mapproj = mapproj, units=units, titles=label)


# PCHO

In [ ]:
units = 'org C [mmol m$^{-3}$]'
label='Polysaccharides {0}-{1} at {2} m'.format(years[0],years[1],depth)

PCHO = pf.get_data(resultpath, "PCHO", years, mesh, runid=runid, how="mean", compute=True, depth=depth)

fig = pf.plot(mesh, PCHO, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Atlantic transect ------------------------------------------------------------------------------
lon_start = -60
lat_start = -80
lon_end = 10
lat_end = 83
npoints   = 50
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

In [ ]:
# load data
PCHOall = pf.get_data(resultpath, "PCHO", years=years, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(PCHO))

print('PCHO loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    years[0],years[1],depth, np.nanmin(PCHOall), np.nanmax(PCHOall), np.nanmean(PCHOall)))

# extract transect data
dist, tfPCHO = pf.get_transect(PCHOall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfPCHO,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='PCHO [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis',
            figsize=(15, 6))

# TEP

In [ ]:
NCfesom = resultpath + '/TEP.fesom.1948.nc'
!ncdump -h $NCfesom

In [ ]:
depth = depth

label='TEP {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'TEP-C [mmol m$^{-3}$]'

TEP = pf.get_data(resultpath, "TEP", years, mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, TEP, mapproj = mapproj, units=units, titles=label)

In [ ]:
# various depths

units = 'TEP-C [mmol m$^{-3}$]'
years1 = [1970,1980]

depth = 0
TEP_0 = pf.get_data(resultpath, "TEP", years1, mesh, runid=runid, how="mean", compute=True, depth=depth)
label_0='TEP {0}-{1} at {2} m'.format(years1[0],years1[1],depth)

depth = 5
TEP_1 = pf.get_data(resultpath, "TEP", years1, mesh, runid=runid, how="mean", compute=True, depth=depth)
label_1='TEP {0}-{1} at {2} m'.format(years1[0],years1[1],depth)

depth = 10
TEP_2 = pf.get_data(resultpath, "TEP", years1, mesh, runid=runid, how="mean", compute=True, depth=depth)
label_2='TEP {0}-{1} at {2} m'.format(years1[0],years1[1],depth)

depth = 20
TEP_3 = pf.get_data(resultpath, "TEP", years1, mesh, runid=runid, how="mean", compute=True, depth=depth)
label_3='TEP {0}-{1} at {2} m'.format(years1[0],years1[1],depth)

In [ ]:
fig = pf.plot(mesh, [TEP_0,TEP_1,TEP_2,TEP_3], mapproj = mapproj, units=units, 
              rowscol = (2,2), figsize = (15,10),
              levels = (0,15,61), #cmap = 'viridis',
              titles=[label_0,label_1,label_2,label_3])

# integrals

In [ ]:
# load nodal area -------------------------------------------------------------------------------------
f = Dataset(resultpath + '/fesom.mesh.diag.nc', 'r')
nod_area = f.variables['nod_area'][:] # Moritz
nod_area_surface = nod_area[0,:]

#print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
   
years2 = np.arange(years[0],years[1]+1,1)
print(years2)
        
# calculating mean TEP per year -------------------------------------------------------------------------------
TEP00glob_total = np.zeros(len(years2))
TEP05glob_total  = np.zeros(len(years2))
TEP10glob_total = np.zeros(len(years2))
#TEP20glob_total   = np.zeros(len(years2))

TEP00ao_total = np.zeros(len(years2))
TEP05ao_total = np.zeros(len(years2))
TEP10ao_total = np.zeros(len(years2))

TEP00so_total = np.zeros(len(years2))
TEP05so_total = np.zeros(len(years2))
TEP10so_total = np.zeros(len(years2))


In [ ]:
## Arctic Oean, N of 70 -------------------------------------------------------------------------------
box=[-180, 180, 70, 90] 
left, right, down, up = box
mask_AO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_AO = nod_area[:,mask_AO]
#nod_area_AO_surface = nod_area[0,mask_AO]

## Souther Oean -------------------------------------------------------------------------------
box=[-180, 180, -90, -50] 
left, right, down, up = box
mask_SO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_SO = nod_area[:,mask_SO]
#nod_area_SO_surface = nod_area[0,mask_SO]
#print(np.shape(nod_area_SO_surface))
        

In [ ]:
for i in range(len(years2)):
    ind_year = int(years2[i])
    print('loading {0}'.format(ind_year))
    # TEP:units = "mmolC/m3"
    
    # 0m ############################
    TEP00glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 0,
                        compute=True, runid='fesom', silent=True)
    TEP00glob_total[i] = np.nanmean(TEP00glob)#*nod_area_surface)

    TEP00ao_total[i] = np.nanmean(TEP00glob[mask_AO])
    #TEP00ao_total[i] = np.nanmean(TEP00ao)
    
    TEP00so_total[i] = np.nanmean(TEP00glob[mask_SO])
    #TEP00so_total[i] = np.nanmean(TEP00so)
    
    # 5 m ############################
    TEP05glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 5,
                        compute=True, runid='fesom', silent=True)

    TEP05glob_total[i] = np.nanmean(TEP05glob)

    TEP05ao_total[i] = np.nanmean(TEP05glob[mask_AO])
    
    TEP05so_total[i] = np.nanmean(TEP05glob[mask_SO])

    # 10 m ############################
    TEP10glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 10,
                        compute=True, runid='fesom', silent=True)

    TEP10glob_total[i] = np.nanmean(TEP10glob)

    TEP10ao_total[i] = np.nanmean(TEP10glob[mask_AO])
    
    TEP10so_total[i] = np.nanmean(TEP10glob[mask_SO])

In [ ]:
# plotting total TEP -------------------------------------------------------------------------------        
fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,5)

# 0 m ############################

plt.subplot(2, 3, 1)
plt.plot(years2,TEP00glob_total,'.-',label='0 m')
plt.plot(years2,TEP05glob_total,'.-',label='5 m')
plt.plot(years2,TEP10glob_total,'.-',label='10 m')
plt.title('Global mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 3, 2)
plt.plot(years2,TEP00ao_total,'.-',label='0 m')
plt.plot(years2,TEP05ao_total,'.-',label='5 m')
plt.plot(years2,TEP10ao_total,'.-',label='10 m')
plt.title('N of 70N mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 3, 3)
plt.plot(years2,TEP00so_total,'.-',label='0 m')
plt.plot(years2,TEP05so_total,'.-',label='5 m')
plt.plot(years2,TEP10so_total,'.-',label='10 m')
plt.title('S. Ocean mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')



# Atlantic Transect

In [ ]:
# get data
TEPall = pf.get_data(resultpath, "TEP", [last_year-10,last_year], mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(TEPall))

print('TEP loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(TEPall), np.nanmax(TEPall), np.nanmean(TEPall)))

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfTEP = pf.get_transect(TEPall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfTEP,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='TEP-C [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis',
            figsize=(15, 6))

# Detritus Carbon

In [ ]:
NCfesom = resultpath + '/DetC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='DetC {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'Detritus C [mmol m$^{-3}$]'

DetC = pf.get_data(resultpath, "DetC", [last_year-10,last_year], mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DetC, mapproj = mapproj, units=units, titles=label)

# extracellular carbon

In [ ]:
NCfesom = resultpath + '/EOC.fesom.1948.nc'
!ncdump -h $NCfesom

#label='TEP {0}-{1} at {2} m'.format(years[0],years[1],depth)
#units = 'TEP-C [mmol m$^{-3}$]'
#TEP = pf.get_data(resultpath, "EOC", years, mesh, runid=runid, how="mean", compute=True, depth=depth)
#fig = pf.plot(mesh, TEP, mapproj = mapproj, units=units, titles=label)

# NPP

In [ ]:
# map of total Diatom NPP 

NCfesom = resultpath + '/NPPd.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mmolC/m2/d to mg C/m2/d

label='Diatom NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPd = pf.get_data(resultpath, "NPPd", 
                   #years, 
                   [last_year-10,last_year],
                   mesh, runid=runid, how="mean", compute=True, silent=True)

#print(np.shape(NPPd))
# NPP is already vertically integrated!

NPPd = NPPd * 12.01
fig = pf.plot(mesh, NPPd, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# map of nanophytoplankton NPP

NCfesom = resultpath + '/NPPn.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mg C/m2/d

label='Nanophytoplankton NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPn = pf.get_data(resultpath, "NPPn", 
                   #years, 
                   [last_year-10,last_year],
                   mesh, runid=runid, how="mean", compute=True,silent=True)

#print(np.shape(NPPn))
# NPP is already vertically integrated!

NPPn = NPPn *12.01

fig = pf.plot(mesh, NPPn, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# total NPP
label='Total Net Primary Production {0}-{1}'.format(last_year-10,last_year)
units = 'Mean total NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPtot = NPPd + NPPn
fig = pf.plot(mesh, NPPtot, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

# DIC

In [ ]:
# DIC

NCfesom = resultpath + '/DIC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Dissolved inorganic Carbon {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'DIC [mmol  m$^{-3}$]'
DIC = pf.get_data(resultpath, "DIC", years, mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DIC, mapproj = mapproj, units=units, titles=label)


# export to html

In [ ]:
%autosave 2

#time.sleep(15)

!jupyter nbconvert MASTER_Script_python3.ipynb --output-dir=$htmlpath --output $htmlname --to html 